In [2]:
#exploring on hard instances full trainset

In [3]:
#cd

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"#imports
#imports
import torch
import torch.nn.functional as F

from torch.nn import Linear


#from kernel.datasets import get_dataset
from itertools import product

import time


from torch import tensor
from torch.optim import Adam
from torch.optim import SGD



from sklearn.model_selection import StratifiedKFold
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader



from math import ceil

from torch.nn import Linear

from torch.distributions import categorical
from torch.distributions import Bernoulli

import torch.nn

%matplotlib inline
from matplotlib import pyplot as plt

#import pygraphviz as pgv

from torch_geometric.utils import convert as cnv
from torch_geometric.utils import sparse as sp

from torch_geometric.data import Data

#import pygraphviz as pgv

from networkx.drawing.nx_agraph import graphviz_layout

import networkx as nx


from torch.utils.data.sampler import RandomSampler

from torch.nn.functional import gumbel_softmax


from torch.distributions import relaxed_categorical

import myfuncs

#from tensorboardX import SummaryWriter

from torch_geometric.nn.inits import uniform
from torch_geometric.nn.inits import glorot, zeros


from torch.nn import Parameter

from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

from torch_geometric.utils import degree

from torch_geometric.nn import GINConv, GATConv, global_mean_pool, NNConv, GCNConv



from torch.nn import Parameter

from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU
from torch_geometric.nn import MessagePassing

import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
from torch_geometric.nn import GINConv, global_mean_pool


from torch_geometric.data import Batch 


from torch_scatter import scatter_min, scatter_max, scatter_add, scatter_mean


from torch import autograd

from torch_geometric.utils import to_dense_batch, to_dense_adj


from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops


from torch_geometric.utils import dropout_adj, to_undirected, to_networkx
from torch_geometric.utils import is_undirected

from cut_utils import get_diracs

import scipy

import scipy.io

from matplotlib.lines import Line2D

from torch_geometric.utils.convert import from_scipy_sparse_matrix

import GPUtil


from networkx.algorithms.approximation import max_clique


import pickle


from torch_geometric.nn import SplineConv, global_mean_pool, DataParallel

from torch_geometric.data import DataListLoader
from random import shuffle


from networkx.algorithms.approximation import max_clique
from networkx.algorithms import graph_clique_number

from networkx.algorithms import find_cliques

from torch_geometric.nn.norm import graph_size_norm


In [5]:
import visdom 
from visdom import Visdom 
import numpy as np
import matplotlib.pyplot as plt




def plot_grad_flow( named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            
    plt.plot(ave_grads, alpha=0.3, color="b")
    plt.yscale('log')
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(ave_grads))
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    

class VisdomLinePlotter(object):
    """Plots to Visdom"""
    def __init__(self, env_name='main'):
        self.viz = Visdom(port=8097)
        self.env = env_name
        self.plots = {}
    def plot(self, var_name, split_name, title_name, x, y):
        if title_name not in self.plots:
            self.plots[title_name] = self.viz.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
                legend=[split_name],
                title=title_name,
                xlabel='Epochs',
                ylabel=var_name
            ))
        else:
            self.viz.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[title_name], name=split_name, update = 'append')
    

    def histog(self,title_name,vals):
        if title_name not in self.plots:
            self.plots[title_name] = self.viz.histogram(X=vals,env=self.env,opts=dict(title=title_name,numbins=20))
        else:
            self.viz.histogram(X=vals,env=self.env,win=self.plots[title_name],opts=dict(title=title_name,numbins=20, update = 'replace'))
  
    def gradflow(self, model, title_name):
                    title_name = "Gradflow"
                    layers = []
                    ave_grads = []
                    for n, p in net.named_parameters():
                            if(p.requires_grad) and ("bias" not in n):
                                     layers.append(str(n))
                                     ave_grads.append(p.grad.abs().mean().cpu().numpy())
                    if title_name not in self.plots:
                        self.plots[title_name]= self.viz.line(X= list(range(len(layers))),Y= np.array(ave_grads),  env = self.env,  opts=dict(fillarea=True, title = title_name, xtick = True,
        xtickmin=0, xtickmax=len(layers), xtickvals = list(range(len(layers))), xtickstep=1/len(layers), xticklabels = layers) )    
                    else:
                        self.viz.line(X= list(range(len(layers))), Y=ave_grads, env = self.env ,win=self.plots[title_name], opts=dict(fillarea=True, title = title_name, xtick = True,
        xtickmin=0, xtickmax=len(layers), xtickvals = list(range(len(layers))), xtickstep=1/len(layers), xticklabels = layers, update="append")) 


In [6]:
class GATAConv(MessagePassing):
    r"""The graph attentional operator from the `"Graph Attention Networks"
    <https://arxiv.org/abs/1710.10903>`_ paper

    .. math::
        \mathbf{x}^{\prime}_i = \alpha_{i,i}\mathbf{\Theta}\mathbf{x}_{i} +
        \sum_{j \in \mathcal{N}(i)} \alpha_{i,j}\mathbf{\Theta}\mathbf{x}_{j},

    where the attention coefficients :math:`\alpha_{i,j}` are computed as

    .. math::
        \alpha_{i,j} =
        \frac{
        \exp\left(\mathrm{LeakyReLU}\left(\mathbf{a}^{\top}
        [\mathbf{\Theta}\mathbf{x}_i \, \Vert \, \mathbf{\Theta}\mathbf{x}_j]
        \right)\right)}
        {\sum_{k \in \mathcal{N}(i) \cup \{ i \}}
        \exp\left(\mathrm{LeakyReLU}\left(\mathbf{a}^{\top}
        [\mathbf{\Theta}\mathbf{x}_i \, \Vert \, \mathbf{\Theta}\mathbf{x}_k]
        \right)\right)}.

    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        heads (int, optional): Number of multi-head-attentions.
            (default: :obj:`1`)
        concat (bool, optional): If set to :obj:`False`, the multi-head
            attentions are averaged instead of concatenated.
            (default: :obj:`True`)
        negative_slope (float, optional): LeakyReLU angle of the negative
            slope. (default: :obj:`0.2`)
        dropout (float, optional): Dropout probability of the normalized
            attention coefficients which exposes each node to a stochastically
            sampled neighborhood during training. (default: :obj:`0`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.
    """

    def __init__(self, in_channels, out_channels, heads=1, concat=True,
                 negative_slope=0.2, dropout=0, bias=True, **kwargs):
        super(GATAConv, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.weight = Parameter(
            torch.Tensor(in_channels, heads * out_channels))
        self.att = Parameter(torch.Tensor(1, heads, 2 * out_channels))

        if bias and concat:
            self.bias = Parameter(torch.Tensor(heads * out_channels))
        elif bias and not concat:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        glorot(self.att)
        zeros(self.bias)


    def forward(self, x, edge_index, size=None):
        """"""
        if size is None and torch.is_tensor(x):
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        if torch.is_tensor(x):
            x = torch.matmul(x, self.weight)
        else:
            x = (None if x[0] is None else torch.matmul(x[0], self.weight),
                 None if x[1] is None else torch.matmul(x[1], self.weight))

        return self.propagate(edge_index, size=size, x=x)


    def message(self, edge_index_i, x_i, x_j, size_i):
        # Compute attention coefficients.
        x_j = x_j.view(-1, self.heads, self.out_channels)
        if x_i is None:
            alpha = (x_j * self.att[:, :, self.out_channels:]).sum(dim=-1)
        else:
            x_i = x_i.view(-1, self.heads, self.out_channels)
            alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)

        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, edge_index_i, size_i)

        # Sample attention coefficients stochastically.
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)

        return x_j * alpha.view(-1, self.heads, 1)

    def update(self, aggr_out):
        if self.concat is True:
            aggr_out = aggr_out.view(-1, self.heads * self.out_channels)
        else:
            aggr_out = aggr_out.mean(dim=1)

        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.heads)



In [7]:
class GCNConv2(MessagePassing):
    r"""The graph convolutional operator from the `"Semi-supervised
    Classfication with Graph Convolutional Networks"
    <https://arxiv.org/abs/1609.02907>`_ paper

    .. math::
        \mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},

    where :math:`\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}` denotes the
    adjacency matrix with inserted self-loops and
    :math:`\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}` its diagonal degree matrix.

    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        improved (bool, optional): If set to :obj:`True`, the layer computes
            :math:`\mathbf{\hat{A}}` as :math:`\mathbf{A} + 2\mathbf{I}`.
            (default: :obj:`False`)
        cached (bool, optional): If set to :obj:`True`, the layer will cache
            the computation of :math:`{\left(\mathbf{\hat{D}}^{-1/2}
            \mathbf{\hat{A}} \mathbf{\hat{D}}^{-1/2} \right)}`.
            (default: :obj:`False`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True):
        super(GCNConv2, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)
        edge_weight = edge_weight.view(-1)
        assert edge_weight.size(0) == edge_index.size(1)

#         print("before: ")
#         print(edge_weight.shape)
#         print(edge_index.shape)
        
        #edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        #edge_index = add_self_loops(edge_index, num_nodes)
#         loop_weight = torch.full((num_nodes, ),
#                                  1 if not improved else 2,
#                                  dtype=edge_weight.dtype,
#                                  device=edge_weight.device)
        #edge_weight = torch.cat([edge_weight, loop_weight], dim=0)
#         print("after: ")
#         print(edge_weight.shape)
#         print(edge_index.shape)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""
        x = torch.matmul(x, self.weight)

        if not self.cached or self.cached_result is None:
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight,
                                         self.improved, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result

        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)


In [8]:
    def propagate(x, edge_index):
        row, col = edge_index
        out = scatter_add(x[col], row, dim=0)
        return out

    def get_mask(x, edge_index, hops):
        for k in range(hops):
            x = propagate(x, edge_index)
        mask = (x>0).float()
        return mask

    
    def total_var(x, edge_index, batch, undirected = True):
        row, col = edge_index
        if undirected:
            tv = (torch.abs(x[row]-x[col])) * 0.5
        else:
            tv = (torch.abs(x[row]-x[col]))
        
        tv = scatter_add(tv, batch[row], dim=0)
        return  tv

In [9]:
#from pytorch_memlab import profile, mem_reporter
#@profile
class cliqueMPNN_hindsight_earlyGAT(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden1, hidden2, deltas, elasticity=0.01, num_iterations = 30):
        super(cliqueMPNN_hindsight_earlyGAT, self).__init__()
        self.hidden1 = hidden1
        self.hidden2 = hidden2
        self.momentum = 0.1
        #self.nns = Sequential(
                #Linear(2*hidden, hidden*hidden),
                #LeakyReLU(0.1))
        self.num_iterations = num_iterations
        self.convs = torch.nn.ModuleList()
        self.deltas = deltas
        self.numlayers = num_layers
        self.elasticity = elasticity
        self.heads = 8
        self.concat = True
        
        self.bns = torch.nn.ModuleList()
        for i in range(num_layers-1):
            self.bns.append(BN(self.heads*self.hidden1, momentum=self.momentum))

        
#         self.nns = torch.nn.ModuleList()
#         for i in range(num_layers-1):
#             self.nns.append(Sequential(
#                 Linear(2*hidden, hidden*hidden),
#                 LeakyReLU(0.1)))
        
        self.convs = torch.nn.ModuleList()        
        for i in range(num_layers - 1):
                #self.convs.append(GATAConv(self.heads*self.hidden1, self.hidden1, concat=self.concat ,heads=self.heads))
                self.convs.append(GINConv(Sequential(
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            BN(self.heads*self.hidden1, momentum=self.momentum),
        ),train_eps=True))
                #self.convs.append(GCNConv2(self.heads*self.hidden1, self.heads*self.hidden1))
        self.bn1 = BN(self.heads*self.hidden1)
        #self.conv1 = GATAConv(self.hidden2, self.hidden1, concat=self.concat ,heads=self.heads)
       
        self.conv1 = GINConv(Sequential(Linear(self.hidden2,  self.heads*self.hidden1),
            ReLU(),
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            BN(self.heads*self.hidden1, momentum=self.momentum),
        ),train_eps=True)

        if self.concat:
            self.lin1 = Linear(self.heads*self.hidden1, self.hidden1)
        else:
            self.lin1 = Linear(self.hidden1, self.hidden1)

        #self.bn2 = BN(self.hidden1, momentum=self.momentum)
        self.lin2 = Linear(self.hidden1, 1)
        self.gnorm = graph_size_norm.GraphSizeNorm()

                    


    def reset_parameters(self):
        self.conv1.reset_parameters()
        #self.conv2.reset_parameters()
        
        for conv in self.convs:
            conv.reset_parameters()
            
        for bn in self.bns:
            bn.reset_parameters()
            
        self.bn1.reset_parameters()
        self.lin1.reset_parameters()
        #self.bn2.reset_parameters()
        self.lin2.reset_parameters()






    def forward(self, data, edge_dropout = None, penalty_coefficient = 0.25):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch
        num_graphs = batch.max().item() + 1
        row, col = edge_index     
        total_num_edges = edge_index.shape[1]
        N_size = x.shape[0]

        
        if edge_dropout is not None:
            edge_index = dropout_adj(edge_index, edge_attr = (torch.ones(edge_index.shape[1], device=device)).long(), p = edge_dropout, force_undirected=True)[0]
            edge_index = add_remaining_self_loops(edge_index, num_nodes = batch.shape[0])[0]
                
        reduced_num_edges = edge_index.shape[1]
        current_edge_percentage = (reduced_num_edges/total_num_edges)
        no_loop_index,_ = remove_self_loops(edge_index)  
        no_loop_row, no_loop_col = no_loop_index

        xinit= x.clone()

        #x = torch.cat([x.unsqueeze(-1),torch.rand((x.shape[0],self.hidden2-1),device=device)], dim=1)
        #x = torch.rand((x.shape[0],self.hidden2),device=device)
        #
        x = x.unsqueeze(-1)
        #mask = get_mask(x,edge_index,1).to(x.dtype)
        
#         print("xbefore:",x.shape)

        mask = get_mask(x,edge_index,1).to(x.dtype)
        x = F.leaky_relu(self.conv1(x, edge_index))# +x
        x = x*mask

#         print("xshape:",x.shape)
#         print("mask: ", mask.shape)
        
        #x = F.leaky_relu(self.conv1(x, edge_index)) + x
        #x = self.conv1(x, edge_index)
        #print(x.shape)
        x = self.gnorm(x)
        x = self.bn1(x)
        

        
        #print(x.shape)
            
        for conv, bn in zip(self.convs, self.bns):
            if(x.dim()>1):
                x =  x+F.leaky_relu(conv(x, edge_index))
                mask = get_mask(mask,edge_index,1).to(x.dtype)
                x = x*mask
                x = self.gnorm(x)
                x = bn(x)


#         x = self.conv2(x, edge_index)
#         mask = get_mask(mask,edge_index,1).to(x.dtype)
#         x = x*mask
        xpostconvs = x.detach()
        #
        x = F.leaky_relu(self.lin1(x)) 
        x = x*mask
        #x = self.gnorm(x)
        #x = self.bn2(x)


        xpostlin1 = x.detach()
        #x = F.dropout(x, p=0.0, training=self.training)
        x = F.leaky_relu(self.lin2(x)) 
       # x = self.gnorm(x)

        x = x*mask


        #xprethresh = x.detach()

        #x = x/scatter_add(torch.abs(x), batch,0)[batch]

        
        #calculate min and max
        batch_max = scatter_max(x, batch, 0, dim_size= N_size)[0]
        batch_max = torch.index_select(batch_max, 0, batch)        
        batch_min = scatter_min(x, batch, 0, dim_size= N_size)[0]
        batch_min = torch.index_select(batch_min, 0, batch)

                
        x = (x-batch_min)/(batch_max+1e-6-batch_min)
        #x = x*mask + mask*1e-6
        

        probs=x
        
        #print(probs.shape)
        
        x2 = x.detach()              
        deg = degree(row).unsqueeze(-1) 
        totalvol = scatter_add(deg.detach()*torch.ones_like(x, device=device), batch, 0)+1e-6
        totalcard = scatter_add(torch.ones_like(x, device=device), batch, 0)+1e-6
        
                
        #volume within receptive field
        #recvol_hard = scatter_add(deg*mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        #reccard_hard = scatter_add(mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        
        #assert recvol_hard.mean()/totalvol.mean() <=1, "Something went wrong! Receptive field is larger than total volume."

        
        
        x2 =  ((x2 - torch.rand_like(x, device = device))>0).float()
        
        vol_1 = scatter_add(probs*deg, batch, 0)+1e-6
        card_1 = scatter_add(probs, batch,0)            
        #rec_field = scatter_add(mask, batch, 0)+1e-6
        set_size = scatter_add(x2, batch, 0)
#         tv_hard = total_var(x2, edge_index, batch)
        vol_hard = scatter_add(deg*x2, batch, 0, dim_size = batch.max().item()+1)+1e-6 
#         conduct_hard = tv_hard/vol_hard
            
#         rec_field_ratio = set_size/rec_field
#         rec_field_volratio = vol_hard/recvol_hard
        total_vol_ratio = vol_hard/totalvol
        
        #volume within receptive field
        #recvol_hard = scatter_add(deg*mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        #reccard_hard = scatter_add(mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        
        
        #calculating the terms for the expected distance between clique and graph
        pairwise_prodsums = torch.zeros(num_graphs, device = device)
        for graph in range(num_graphs):
            batch_graph = (batch==graph)
            pairwise_prodsums[graph] = (torch.conv1d(probs[batch_graph].unsqueeze(-1), probs[batch_graph].unsqueeze(-1))).sum()/2
        
        
        ##############CAREFUL
        self_sums = scatter_add((probs*probs), batch, 0, dim_size = num_graphs)/2.

        #self_sums = scatter_add((probs*probs), batch, 0, dim_size = num_graphs)/1.
        
        #expected_weight_G = scatter_add(probs[no_loop_row]*probs[no_loop_col], batch[no_loop_row], 0, dim_size = num_graphs)/2
        
        expected_weight_G = scatter_add(probs[no_loop_row]*probs[no_loop_col], batch[no_loop_row], 0, dim_size = num_graphs)/2.
        expected_clique_weight = (pairwise_prodsums.unsqueeze(-1) - self_sums)/1.
        expected_distance = (expected_clique_weight - expected_weight_G)
        
        
        #lambda_factors = (torch.rand((30,1), device=device))*penalty_coefficient-0.10
        
        
        #hindsight = torch.ones_like(lambda_factors)*expected_distance.unsqueeze(-1)*0.5 - lambda_factors*expected_weight_G.unsqueeze(-1)
        

       # print(expected_clique_weight.shape)
        #print(self_sums.shape)

       # expected_loss =  #torch.median(hindsight, 1)[0]
        #print(expected_loss.shape)
        
        max_set_weight = (scatter_add(torch.ones_like(x)[no_loop_row], batch[no_loop_row], 0, dim_size = num_graphs)/2).squeeze(-1)
        
        #print("how many:", (data.batch==0).sum())
        
        set_weight = (scatter_add(x2[no_loop_row]*x2[no_loop_col], batch[no_loop_row], 0, dim_size = num_graphs)/2)+1e-6
        clique_edges_hard = (set_size*(set_size-1)/2) +1e-6
        clique_dist_hard = set_weight/clique_edges_hard
        
        
#          print("cardinalities: ", cardinalities)
#         print("max_set_weight:", max_set_weight)
#        print("max possible weight: ", max_possible_weight)
        #print(max_set_weight.shape)
        #print(max_possible_weight.shape)
        cardinalities =  scatter_add(torch.ones_like(data.batch),data.batch,dim=0).float()
        max_possible_weight = (cardinalities*(cardinalities-1)/2)
#         diff = (max_possible_weight - max_set_weight) 
        penalty_coefficient =   (max_set_weight/max_possible_weight) * penalty_coefficient
        #penalty_coefficient =  penalty_coefficient*(clique_dist_hard/(1-clique_dist_hard))
                
        #penalty_coefficient[clique_dist_hard>0.95] = 1.

        #print("pen coeff:", penalty_coefficient)
#         print("Diff: ", diff)
#         print("penalties:", penalties.shape)
#         print("exp dist:", expected_distance.shape)
        
        #assert ((max_possible_weight<max_set_weight).sum())<=1e-6, "Invalid calculation"
        expected_ratio = expected_weight_G/expected_distance
        expected_ratio = expected_ratio.detach()
        #penalty_coefficient = expected_ratio*penalty_coefficient
       # print(expected_ratio.shape)
                               
        #expected_loss = (penalty_coefficient)*expected_distance*0.5*expected_weight_G - 0.5*expected_weight_G
        #expected_loss = expected_clique_weight/expected_weight_G
        expected_loss = (penalty_coefficient)*expected_distance*0.5 - 0.5*expected_weight_G
        #expected_loss = (penalty_coefficient)*expected_clique_weight*0.5- 1.*expected_weight_G
        
#         print("pen coeff: ", penalty_coefficient)
#         print("loss: ", expected_loss)
    
    
#         for iter_graph in range(num_graphs):
#             if clique_dist_hard[iter_graph]<0.5:
#                 print("problematic graph exp distance: ", expected_distance[iter_graph])
#                 print("problematic graph exp cardinality : ", card_1[iter_graph])

#                 print("problematic graph num nodes: ", totalcard[iter_graph])
                
        clique_check = ((clique_edges_hard != clique_edges_hard))
        
        
        setedge_check  = ((set_weight != set_weight))
        
        
        
        assert ((clique_dist_hard>=1.1).sum())<=1e-6, "Invalid set vol/clique vol ratio."
        
#        print(expected_loss.shape)
        
       # normalize = cardinalities.max()/cardinalities
        #print(f"normalize: {normalize}")
        loss = expected_loss#*normalize
        #loss = expected_loss*(1/(clique_dist_hard+0.1))


        retdict = {}
        
        retdict["output"] = [probs.squeeze(-1),"hist"]   #output
        #retdict["clique_check"] = [clique_edges_hard, "hist"]
        #retdict["set_weight_check"] = [set_weight, "hist"]
        #retdict["|Expected_vol - Target|"]= [targetcheck.squeeze(-1), "hist"] #absolute distance from targetvol
        #retdict["Expected_volume"] = [vol_1.mean(),"sequence"] #volume
        retdict["Expected_cardinality"] = [card_1.mean(),"sequence"]
        retdict["Expected_cardinality_hist"] = [card_1,"hist"]
        retdict["losses histogram"] = [loss.squeeze(-1),"hist"]
        retdict["Set sizes"] = [set_size.squeeze(-1),"hist"]
        retdict["volume_hard"] = [vol_hard.mean(),"aux"] #volume2
        retdict["cardinality_hard"] = [set_size[0],"sequence"] #volumeq
        retdict["Expected weight(G)"]= [expected_weight_G.mean(), "sequence"]
        retdict["Expected maximum weight"] = [expected_clique_weight.mean(),"sequence"]
        retdict["Expected distance"]= [expected_distance.mean(), "sequence"]
        #retdict["cut1"] = [tv.mean(),"sequence"] #cut1
        #retdict["cut_hard"] = [tv_hard.mean(),"sequence"] #cut1
        #retdict["Average cardinality ratio of receptive field "] = [rec_field_ratio.mean(),"sequence"] 
        #retdict["Recfield volume/Total volume"] = [recvol_hard.mean()/totalvol.mean(), "sequence"]
        #retdict["Average ratio of receptive field volume"]= [rec_field_volratio.mean(),'sequence']
        retdict["Currvol/Cliquevol"] = [clique_dist_hard.mean(),'sequence']
        retdict["Currvol/Cliquevol all graphs in batch"] = [clique_dist_hard.squeeze(-1),'hist']
        retdict["Average ratio of total volume"]= [total_vol_ratio.mean(),'sequence']
        #retdict["penalty coffs"] = [penalty_coefficient.squeeze(-1),"hist"]
        retdict["cardinalities"] = [cardinalities.squeeze(-1),"hist"]
        retdict["Current edge percentage"] = [torch.tensor(current_edge_percentage),'sequence']
        #retdict["Clique Weight hard"] = [clique_edges_hard[0], 'sequence']
        #retdict["Set weight"] = [set_weight[0], 'sequence']
        #retdict["mask"] = [mask, "aux"] #mask
        #retdict["xinit"] = [xinit,"hist"] #layer input diracs
        #retdict["xpostlin1"] = [xpostlin1.mean(1),"hist"] #after first linear layer
        #retdict["xprethresh"] = [xprethresh.mean(1),"hist"] #pre thresholding activations 195 x 1
        #retdict["xsoftbin"] = [xsoftbin.mean(1),"hist"] #soft binarized output
        #retdict["lossvol"] = [lossvol.mean(),"sequence"] #volume constraint
        #retdict["losscard"] = [losscard.mean(),"sequence"] #cardinality constraint
        retdict["loss"] = [loss.mean().squeeze(),"sequence"] #final loss

        return retdict
    
    def __repr__(self):
        return self.__class__.__name__


In [10]:
def predict(model, data_loader, recfield):
    model.train()
    avg_loss = 0
    avg_cliqdist = 0
    exp_cardinalities = torch.tensor(0)
    for data in data_loader:
        optimizer.zero_grad()
        data = data.to(device)
        #print("data  batchsum: ",(data.batch==1).sum())
        data = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = recfield)
        data = data.to(device)
        #print("data prime batchsum: ",(data.batch==1).sum())
        retdict = model(data)
        avg_loss += retdict['loss'][0].item()/len(data_loader)
        avg_cliqdist += retdict["Currvol/Cliquevol"][0].item()/len(data_loader)
        exp_cardinalities = [retdict["Expected_cardinality_hist"][0]]
    #print("retdict: ", retdict["Expected_cardinality_hist"][0] )
    return avg_loss, avg_cliqdist, exp_cardinalities 
        

In [11]:
import gurobipy as gp
from gurobipy import GRB

def solve_gurobi_maxclique(nx_graph, time_limit = None):

    nx_complement = nx.operators.complement(nx_graph)
    x_vars = {}
    m = gp.Model("mip1")
    m.params.OutputFlag = 0

    if time_limit:
        m.params.TimeLimit = time_limit

    for node in nx_complement.nodes():
        # Create a new model

        # Create variables
        x_vars['x_'+str(node)] = m.addVar(vtype=GRB.BINARY, name="x_"+str(node))

    count_edges = 0
    for edge in nx_complement.edges():
        m.addConstr(x_vars['x_'+str(edge[0])] + x_vars['x_'+str(edge[1])] <= 1,'c_'+str(count_edges))
        count_edges+=1
    # Set objective
    m.setObjective(sum([x_vars['x_'+str(node)] for node in nx_complement.nodes()]), GRB.MAXIMIZE);


    # Optimize model
    m.optimize();

# for v in m.getVars():
#     print('%s %g' % (v.varName, v.x))

# print('Obj: %g' % m.objVal)
    set_size = m.objVal;
    x_vals = [var.x for var in m.getVars()] 

    return set_size, x_vals


In [12]:
datasets = ["TWITTER_SNAP", "COLLAB", "IMDB-BINARY"]
dataset_name = datasets[0]
#datasetname = "COLLAB_shuffle_1"
#datasetname = "TWITTER_SNAP"
#dataset_name = "IMDB-BINARY"

if dataset_name == "TWITTER_SNAP":
    stored_dataset = open('datasets/TWITTER_SNAP.p', 'rb')        
elif dataset_name == "COLLAB":
    stored_dataset = open('datasets/dataset_shuffle_1'+'.p', 'rb')
elif dataset_name == "IMDB-BINARY":
    stored_dataset = open('datasets/IMDB_BINARY.p', 'rb')
#dataset = pickle.load(stored_dataset)


dataset = pickle.load(stored_dataset)

dataset_scale = 1
total_samples = int(np.floor(len(dataset)*dataset_scale))
dataset = dataset[:total_samples]

num_trainpoints = int(np.floor(0.6*len(dataset)))
num_valpoints = int(np.floor(num_trainpoints/3))
num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)


traindata= dataset[0:num_trainpoints]
valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]
testdata = dataset[num_trainpoints + num_valpoints:]

batch_size = 32

train_loader = DataLoader(traindata, batch_size, shuffle=True)
test_loader = DataLoader(testdata, batch_size, shuffle=False)
val_loader =  DataLoader(valdata, batch_size, shuffle=False)


#set up random seeds 
torch.manual_seed(1)
np.random.seed(2)   
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [13]:
# #traindata_x = [data for data in traindata if data.x.shape[0]>400]
# traindata_c = traindata
# train_loader = DataLoader(traindata, batch_size, shuffle=True)
!cd datasets

In [14]:
#### epochs=1000
#number of propagation layers
numlayers=5
elasticity = 0.2

#number of propagation layers + attention
receptive_field= numlayers + 1


val_losses = []
cliq_dists = []

net =  cliqueMPNN_hindsight_earlyGAT(dataset,numlayers, 32, 32,1, elasticity = elasticity)
#net =  cutTMPNN(dataset,numlayers, 64,1)

#envname = "Hard instance experiments" + str(net)+ 'PROTOTYPING_' + datasetname + '_seed: '+ str(r_seed) +"_width1: "+ str(hidden_1) + "_batch size : " + str(batch_size) + "Finalize_Samples_new_code_GCAT" + "_LR: " +str(learning_rate) + "_PenaltyCoeff:  "+ str(penalty_coeff)+ "_Layers: "+ str(numlayers) 

#net =  DataParallel(net)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lr_decay_step_size = 5
lr_decay_factor = 0.95

net.to(device).reset_parameters()
#net = net.to(device)

optimizer = Adam(net.parameters(), lr=0.001, weight_decay=0.00)


In [15]:
#LOAD MODEL INSTEAD OF TRAINING
# net =  cliqueMPNN(dataset,numlayers, 64, 64,1, elasticity = elasticity)

# file_name = '/home/karalias/myrepos/CliqueMPNN/cliqueMPNN_randomround_samples/Models/'+ str(datasetname)+ '/' + str(net)+ '_'+ str(75)+'_samples.pt'
# cpoint = torch.load(file_name)
# net.load_state_dict(cpoint[file_name], strict=False)

# net.to(device).reset_parameters()

In [16]:
#THEEEEEEEEEEEEEEEEEESEEEEEEEEEEEEE ONES WWWWWWWWWWWWWWWWOOOOOOOOOOOOORKKKKKKKKKKK
# epochs = 200
# net.train()
# retdict = {}
# edge_drop_p = 0.0
# edge_dropout_decay = 0.90
# penalty_coeff = 9.00
# penalty_increase = -0.00
# validation_timeout = 500

b_sizes = [32]
l_rates = [0.001]
depths = [4]
coefficients = [4.]
rand_seeds = [66]
widths = [64]

epochs = 100
net.train()
retdict = {}
edge_drop_p = 0.0
edge_dropout_decay = 0.90
penalty_coeff = 9.00
penalty_increase = -0.00
validation_timeout = 75

# b_sizes = [32]
# l_rates = [0.001]
# depths = [3]
# coefficients = [3.5]
# rand_seeds = [68]
# widths = [64]

#THEBEST
# epochs = 100
# net.train()
# retdict = {}
# edge_drop_p = 0.0
# edge_dropout_decay = 0.90
# penalty_coeff = 9.00
# penalty_increase = -0.00
# validation_timeout = 75



for batch_size, learning_rate, numlayers, penalty_coeff, r_seed, hidden_1 in product(b_sizes, l_rates, depths, coefficients, rand_seeds, widths):
   
    torch.manual_seed(r_seed)


    train_loader = DataLoader(traindata, batch_size, shuffle=True)
    test_loader = DataLoader(testdata, batch_size, shuffle=False)
    val_loader =  DataLoader(valdata, batch_size, shuffle=False)

    receptive_field= numlayers + 1
    val_losses = []
    cliq_dists = []

    #hidden_1 = 128
    hidden_2 = 1

    net =  cliqueMPNN_hindsight_earlyGAT(dataset,numlayers, hidden_1, hidden_2 ,1, elasticity = elasticity)
    #net =  cutTMPNN(dataset,numlayers, 64,1)

    #net =  DataParallel(net)

    net.to(device).reset_parameters()
    #net = net.to(device)
    optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.00000)

    envname = "August" + str(net)+ 'PROTOTYPING_' + dataset_name + '_seed: '+ str(r_seed) +"_width1: "+ str(hidden_1) + "_batch size : " + str(batch_size) + "Finalize_Samples_new_code_GCAT" + "_LR: " +str(learning_rate) + "_PenaltyCoeff:  "+ str(penalty_coeff)+ "_Layers: "+ str(numlayers) 
    
    plotter = VisdomLinePlotter(env_name=envname)
    for epoch in range(epochs):
        totalretdict = {}
        count=0
        if epoch % 5 == 0:
            edge_drop_p = edge_drop_p*edge_dropout_decay
            print("Edge_dropout: ", edge_drop_p)

        if epoch % 10 == 0:
            penalty_coeff = penalty_coeff + 0.
            print("Penalty_coefficient: ", penalty_coeff)

        # if epoch % 15 == 0:
        #     val_l, cliq_dist, val_cardinalities= predict(net, val_loader, receptive_field)
        #     #val_cardinalities = torch.tensor(val_cardinalities.detach().cpu())
        #     #print(val_cardinalities)
        #     val_losses += [val_l]
        #     cliq_dists += [cliq_dist]
            # if epoch>30:
            #     plotter.plot("validation loss", "validation loss", "validation loss", epoch, val_losses[-1])
            #     plotter.plot("validation clique dist", "validation clique dist", "validation clique dist", epoch, cliq_dists[-1])
            #     #plotter.histog("validation expected cardinalities",val_cardinalities[0])

            #     if (val_losses[-1] > val_losses[-2]) and (val_losses[-1] > val_losses[-3]) and (epoch>validation_timeout):
            #         print("Converged!")
            #         if val_losses[-2] < val_losses[-3]:
            #             print("Best model epoch: ", epoch-15)
            #             file_name = '/home/karalias/myrepos/CliqueMPNN/cliqueMPNN_randomround_samples/Models/'+ str(dataset_name)+ '/' + str(net)+ '_'+ str(epoch-15)+'_samples.pt'
            #             net.load_state_dict(torch.load(file_name), strict=False)
            #         else:
            #             print("Best model epoch: ", epoch-30)
            #             file_name = '/home/karalias/myrepos/CliqueMPNN/cliqueMPNN_randomround_samples/Models/'+ str(dataset_name)+ '/' + str(net)+ '_'+ str(epoch-30)+'_samples.pt'
            #             net.load_state_dict(torch.load(file_name), strict=False)
            #         break
            #     if epoch%15==0:
            #         file_name = '/home/karalias/myrepos/CliqueMPNN/cliqueMPNN_randomround_samples/Models/'+ str(dataset_name)+ '/' + str(net)+ '_'+ str(epoch)+'_samples.pt'
            #         print("file_name: ", file_name)
            #         torch.save({file_name : net.state_dict()},  file_name)

        print("here2")
        #learning rate schedule
        if epoch % lr_decay_step_size == 0:
            for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_decay_factor * param_group['lr']

        #show currrent epoch and GPU utilizationss
        print('Epoch: ', epoch)
        GPUtil.showUtilization()



        #print("here3")


        net.train()
        for data in train_loader:
            count += 1 
            optimizer.zero_grad(), 
            data = data.to(device)
            data_prime = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = receptive_field)

            data = data.to('cpu')
            data_prime = data_prime.to(device)


            retdict = net(data_prime, edge_drop_p, penalty_coeff)
            #print(retdict)

            for key,val in retdict.items():
                if "sequence" in val[1]:
                    if key in totalretdict:
                        totalretdict[key][0] += val[0].item()
                    else:
                        totalretdict[key] = [val[0].item(),val[1]]

                elif count == 1 and "hist" in val[1]: 
                    try:
                        plotter.histog(key, val[0].reshape(-1))
                    except:    
                        print(f"Remember, you need more than 1 value for a histogram! Key: {key}", val[0].shape)


            #print("here4")

            if epoch > 2:
#                     print("pre backward------------------------")
#                     #reporter.report()
#                     print("after backward----------------------")
                    retdict["loss"][0].backward()
                    #reporter.report()

                    torch.nn.utils.clip_grad_norm_(net.parameters(),1)
                    optimizer.step()
                    del(retdict)
                    if count==1:
                          plotter.gradflow(net, "Gradflow")

        if epoch > -1:        
            for key,val in totalretdict.items():
                if "sequence" in val[1]:
                    val[0] = val[0]/(len(train_loader.dataset)/batch_size)
                    plotter.plot(key, key, key, epoch, val[0])

            del data_prime

Setting up a new session...
Edge_dropout:  0.0
Penalty_coefficient:  4.0
here2
Epoch:  0
| ID | GPU | MEM |
------------------
|  0 |  0% | 14% |
|  1 |  0% |  0% |
here2
Epoch:  1
| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
here2
Epoch:  2
| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
here2
Epoch:  3
| ID | GPU | MEM |
------------------
|  0 |  7% | 22% |
|  1 |  0% |  0% |
here2
Epoch:  4
| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
Edge_dropout:  0.0
here2
Epoch:  5
| ID | GPU | MEM |
------------------
|  0 | 10% | 22% |
|  1 |  0% |  0% |
here2
Epoch:  6
| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
here2
Epoch:  7
| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
here2
Epoch:  8
| ID | GPU | MEM |
------------------
|  0 | 12% | 23% |
|  1 |  0% |  0% |
here2
Epoch:  9
| ID | GPU | MEM |
------------------
|  0 |  0% | 24% |
|  1 | 

In [52]:
def get_diracs_2(data, N , sparse = False, flat = False, replace = True, receptive_field = 7, effective_volume_range = 0.1, max_iterations=20):
    device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

    if not sparse:
        graphcount =data.num_nodes #number of graphs in data/batch object
        totalnodecount = data.x.shape[1] #number of total nodes for each graph 
        actualnodecount = 0 #cumulative number of nodes
        diracmatrix= torch.zeros((graphcount,totalnodecount,N),device=device) #matrix with dirac pulses


        for k in range(graphcount):
            graph_nodes = data.mask[k].sum() #number of nodes in the graph
            actualnodecount += graph_nodes #might not need this, we'll see
            probabilities= torch.ones((graph_nodes.item(),1),device=device)/graph_nodes #uniform probs
            node_distribution=OneHotCategorical(probs=probabilities.squeeze())
            node_sample= node_distribution.sample(sample_shape=(N,))
            node_sample= torch.cat((node_sample,torch.zeros((N,totalnodecount-node_sample.shape[1]),device=device)),-1) #concat zeros to fit dataset shape
            diracmatrix[k,:]= torch.transpose(node_sample,dim0=-1,dim1=-2) #add everything to the final matrix
            
        return diracmatrix
    
    else:
            if not is_undirected(data.edge_index):
                data.edge_index = to_undirected(data.edge_index)
                
            original_batch_index = data.batch
            original_edge_index = add_remaining_self_loops(data.edge_index, num_nodes = data.batch.shape[0])[0]
            #original_edge_index, _, node_mask = remove_isolated_nodes(original_edge_index)
            #batch_index = original_batch_index[node_mask]
            batch_index = original_batch_index
            
            graphcount = data.num_graphs #number of graphs in data/batch object
            diracmatrix = torch.zeros(0,device=device)
            batch_prime = torch.zeros(0,device=device).long()
            locationmatrix = torch.zeros(0,device=device).long()
            
            r,c = original_edge_index
            
            
            global_offset = 0
            all_nodecounts = scatter_("add", torch.ones_like(batch_index,device='cpu'), batch_index)
            recfield_vols = torch.zeros(graphcount,device=device)
            total_vols = torch.zeros(graphcount,device=device)
            

            for k in range(graphcount):
                #get edges of current graph, remember to subtract offset
                graph_nodes = all_nodecounts[k]
                if graph_nodes==0:
                    print("all nodecounts: ", all_nodecounts)
                graph_edges = (batch_index[r]==k)
                graph_edge_index = original_edge_index[:,graph_edges]-global_offset           
                gr, gc = graph_edge_index
                
#                 print("Gr: ", gr)
#                 print("Graph edge index: ", graph_edge_index)
#                 print("gr max: ", gr.max())
                
                
                
                #get dirac
                randInt = np.random.choice(range(graph_nodes), N, replace = replace)
                node_sample = torch.zeros(N*graph_nodes,device='cpu')
                offs  = torch.arange(N, device=device)*graph_nodes
                dirac_locations = (offs + torch.from_numpy(randInt).to(device))
                node_sample[dirac_locations] = 1
                
                                
                #calculate receptive field volume and compare to total volume
                mask = get_mask(node_sample, graph_edge_index.detach(), receptive_field).float()

                deg_graph = degree(gr, (graph_nodes.item()))
                
                
                total_volume = deg_graph.sum()
                recfield_volume = (mask*deg_graph).sum()
                volume_range = recfield_volume/total_volume
                total_vols[k] = total_volume
                recfield_vols[k] = recfield_volume
                
                
                #if receptive field volume is less than x% of total volume, resample
                for iteration in range(max_iterations):  
                    randInt = np.random.choice(range(graph_nodes), N, replace = replace)
                    node_sample = torch.zeros(N*graph_nodes,device=device)
                    offs  = torch.arange(N, device=device)*graph_nodes
                    dirac_locations = (offs + torch.from_numpy(randInt).to(device))
                    node_sample[dirac_locations] = 1
                    
                    mask = get_mask(node_sample, graph_edge_index, receptive_field).float()
                    recfield_volume = (mask*deg_graph).sum()
                    volume_range = recfield_volume/total_volume

                    if volume_range > effective_volume_range:
                        recfield_vols[k] = recfield_volume
                        total_vols[k] = total_volume
                        break;
                
                
                dirac_locations2 = torch.from_numpy(randInt).to(device) + global_offset
                global_offset += graph_nodes
                
                diracmatrix = torch.cat((diracmatrix, node_sample),0)
                locationmatrix = torch.cat((locationmatrix, dirac_locations2),0)
             
                
            
                #for batch prime
#                 dirac_indices = torch.arange(N, device=device).unsqueeze(-1).expand(-1, graph_nodes).contiguous().view(-1)
#                 dirac_indices = dirac_indices.long()
#                 dirac_indices += k*N
#                 batch_prime = torch.cat((batch_prime, dirac_indices))



            locationmatrix = diracmatrix.nonzero()


            return Batch(batch = batch_index, x = diracmatrix, edge_index = original_edge_index,
                         y = data.y, locations = locationmatrix, volume_range = volume_range, recfield_vol = recfield_vols, total_vol = total_vols)


In [16]:
def derandomize_clique_final(data, probabilities, draw=False, weight_factor = 0.35, clique_number_bounds = None ,fig = None, device = 'cpu'):
       
    row, col = data.edge_index
    sets = probabilities.detach().unsqueeze(-1)
    batch = data.batch
    
    no_loop_index,_ = remove_self_loops(data.edge_index)        
    no_loop_row, no_loop_col = no_loop_index
    num_graphs = batch.max().item() + 1
    
    

    total_index = 0

    for graph in range(num_graphs):
       #  print("NEW GRAPH-----------------------------------")
            #select subset of nodes and edges corresponding to graph
        mark_edges = batch[no_loop_row] == graph
        nlr_graph, nlc_graph = no_loop_index[:,mark_edges]
        nlr_graph = nlr_graph - total_index
        nlc_graph = nlc_graph - total_index
        batch_graph = (batch==graph)
        graph_probs = sets[batch_graph].detach()
        sorted_inds = torch.argsort(graph_probs.squeeze(-1), descending=True)
        
         #print(batch_graph)
        
        pairwise_prodsums = torch.zeros(1, device = device)
        pairwise_prodsums = (torch.conv1d(graph_probs.unsqueeze(-1), graph_probs.unsqueeze(-1))).sum()/2
        
        self_sums = (graph_probs*graph_probs).sum()


            
        num_nodes = batch_graph.float().sum().item()
        
        current_set_cardinality = 0
        
        for node in range(int(num_nodes)):
        #while current_set_cardinality <= 0:
            #print("Current node: ", node)
            ind_i = total_index + sorted_inds[node]
            graph_probs_0 = sets[batch_graph].detach()
            graph_probs_1 = sets[batch_graph].detach()
            
            graph_probs_0[sorted_inds[node]] = 0
            graph_probs_1[sorted_inds[node]] = 1

            pairwise_prodsums_0 = torch.zeros(1, device = device)
            pairwise_prodsums_0 = (torch.conv1d(graph_probs_0.unsqueeze(-1),graph_probs_0.unsqueeze(-1))).sum()/2

            self_sums_0 = (graph_probs_0*graph_probs_0).sum()

            expected_weight_G_0 = (graph_probs_0[nlr_graph]*graph_probs_0[nlc_graph]).sum()/2
            expected_clique_weight_0 = (pairwise_prodsums_0 - self_sums_0)
            clique_dist_0 = weight_factor*0.5*(expected_clique_weight_0 - expected_weight_G_0)-expected_weight_G_0


            pairwise_prodsums_1 = torch.zeros(1, device = device)
            pairwise_prodsums_1 = (torch.conv1d(graph_probs_1.unsqueeze(-1),graph_probs_1.unsqueeze(-1))).sum()/2

            self_sums_1 = (graph_probs_1*graph_probs_1).sum()

            expected_weight_G_1 = (graph_probs_1[nlr_graph]*graph_probs_1[nlc_graph]).sum()/2
            expected_clique_weight_1 = (pairwise_prodsums_1 - self_sums_1)
            clique_dist_1 = weight_factor* 0.5*(expected_clique_weight_1 - expected_weight_G_1)-expected_weight_G_1


#              #CALCULATE NUMBER OF EDGES AND CONFIRM IF CLIQUE
#             print("clique_dist_0: ", clique_dist_0)
#             print("clique_dist_1: ", clique_dist_1)

            if clique_dist_0 >= clique_dist_1: 
                decided = (graph_probs_1==1).float()
                current_set_cardinality = decided.sum().item()
                current_set_max_edges = (current_set_cardinality*(current_set_cardinality-1))/2
                current_set_edges = (decided[nlr_graph]*decided[nlc_graph]).sum()/2

                if (current_set_edges != current_set_max_edges):
                    sets[ind_i] = 0 #IF NOT A CLIQUE
                else:
                    sets[ind_i] = 1 #IF A CLIQUE
                    

            else:
                   sets[ind_i] = 0


        if draw: 
            dirac = data.locations[graph].item() - total_index
            if fig is None:
                 f1 = plt.figure(graph,figsize=(16,9)) 
            else:
                 f1 = fig
            ax1 = f1.add_subplot(121)
            g1,g2 = drawGraphFromData(data.to('cpu'), graph, vals=sets.squeeze(-1).detach().cpu(), dense=False,seed=dirac, nodecolor=True,edgecolor=False,seedhops=True,hoplabels=True,binarycut=False)
            ax2 = f1.add_subplot(122)
            g1,g2 = drawGraphFromData(data.to('cpu'), graph, vals=probabilities.detach().cpu(), dense=False,seed=dirac, nodecolor=True,edgecolor=False,seedhops=True,hoplabels=True,binarycut=False, clique = True)

            clique_size = len(list(max_clique(g1)))
#              print("NX Clique size is: ", clique_size)
#              print("True clique number: ", graph_clique_number(g1))
        
        
        total_index += num_nodes
        

    expected_weight_G = scatter_add(sets[no_loop_col]*sets[no_loop_row], batch[no_loop_row], 0, dim_size = num_graphs)
    set_cardinality = scatter_add(sets, batch, 0 , dim_size = num_graphs)
    return sets, expected_weight_G.detach(), set_cardinality

In [19]:
#Fast derandomizer
def derandomize_clique_final_speed(data, probabilities, draw=False, weight_factor = 0.35, clique_number_bounds = None ,fig = None, device = 'cpu', beam = 40):
       
    row, col = data.edge_index
    sets = probabilities.detach().unsqueeze(-1)
    blank_sets = torch.zeros_like(probabilities)
    batch = data.batch
    
    no_loop_index,_ = remove_self_loops(data.edge_index)        
    no_loop_row, no_loop_col = no_loop_index
    num_graphs = batch.max().item() + 1
    
    max_cardinalities = torch.zeros(num_graphs)

    total_index = 0

    for graph in range(num_graphs):
        #torch.cuda.memory_summary()
        #print("NEW GRAPH-----------------------------------")
            #select subset of nodes and edges corresponding to graph
        mark_edges = batch[no_loop_row] == graph
        nlr_graph, nlc_graph = no_loop_index[:,mark_edges]
        nlr_graph = nlr_graph - total_index
        nlc_graph = nlc_graph - total_index
        batch_graph = (batch==graph)
        graph_probs = sets[batch_graph].detach()
        sorted_inds = torch.argsort(graph_probs.squeeze(-1), descending=True)

        
         #print(batch_graph)
#         pairwise_prodsums = torch.zeros(1, device = device)
#         pairwise_prodsums = (torch.conv1d(graph_probs.unsqueeze(-1), graph_probs.unsqueeze(-1))).sum()/2
#         self_sums = (graph_probs*graph_probs).sum()
        num_nodes = batch_graph.long().sum()
        #print(num_nodes)
        
        current_set_cardinality = 0
        target_neighborhood = torch.tensor([])
        #final_set = []
        node = 0
        max_width = beam
        if num_nodes>max_width:
            beam_width = max_width 
        else:
            beam_width = num_nodes
            
        max_beam_weight = 0
        max_weight_node = 0
        graph_probs_1 = sets[batch_graph].detach()
        max_cardinality = 0
        
        for node in range(beam_width):
            blank_sets[batch_graph] = 0
            current_set_cardinality = 0

        #for node in range(int(num_nodes)):
        #while current_set_cardinality <= 0:
            #print("Current node here: ", current_set_cardinality)
            ind_i = total_index + sorted_inds[node]

            ind_i = total_index + sorted_inds[node]
            blank_sets[ind_i] = 1
            sets[ind_i] = 1 #IF A CLIQUE=
            #final_set += [ind_i]
            target_neighborhood = torch.unique(nlc_graph[nlr_graph == sorted_inds[node]])
            decided = blank_sets[batch_graph]
            #current_set_cardinality = decided.sum()
            #print("current card: ", current_set_cardinality)
            current_set_max_edges = (current_set_cardinality*(current_set_cardinality-1))/2
            current_set_edges = (decided[nlr_graph]*decided[nlc_graph]).sum()/2
            current_set_cardinality += 1

            neighborhood_probs =  graph_probs[target_neighborhood]
            #print(neighborhood_probs.shape)
            neigh_inds = torch.argsort(neighborhood_probs.squeeze(-1), descending=True)
            #if not neigh_inds:
            sorted_target_neighborhood = target_neighborhood[neigh_inds]


            for node_2 in sorted_target_neighborhood:
            #    print("here")
                ind_i  = total_index + node_2
                #graph_probs_0 = sets[batch_graph].detach()
                #graph_probs_1 = sets[batch_graph].detach()

    #             graph_probs_0[node] = 0
    #             #graph_probs_1[node] = 1

    #             pairwise_prodsums_0 = torch.zeros(1, device = device)

                blank_sets[ind_i] = 1
                sets[ind_i] = 1 #IF A CLIQUE
                current_set_cardinality += 1
                decided = blank_sets[batch_graph]
                #current_set_cardinality = decided.sum()
                current_set_max_edges = (current_set_cardinality*(current_set_cardinality-1))/2
                current_set_edges = (decided[nlr_graph]*decided[nlc_graph]).sum()/2

                if (current_set_edges != current_set_max_edges):
    #                 print("current edges: ", current_set_edges)
    #                 print("current max edges: ", current_set_max_edges)

                    sets[ind_i] = 0 #IF NOT A CLIQUE
                    blank_sets[ind_i] = 0  
                    current_set_cardinality =  current_set_cardinality - 1

            if current_set_cardinality > max_cardinality:
                max_cardinality = current_set_cardinality
            #blank_sets[ind_i] = 0

        max_cardinalities[graph] = max_cardinality
    #             else:
    #                 sets[ind_i] = 1 #IF A CLIQUE
    #                 blank_sets[ind_i] = 1
    #                 #final_set += [ind_i]
    #                 #target_neighborhood = torch.unique(nlc_graph[nlr_graph == ind_i])
    #                 #print(target_neighborhood)

    #             else:
    #                    sets[ind_i] = 0

 
    
    
        if draw: 
            dirac = data.locations[graph].item() - total_index
            if fig is None:
                 f1 = plt.figure(graph,figsize=(16,9)) 
            else:
                 f1 = fig
            ax1 = f1.add_subplot(121)
            g1,g2 = drawGraphFromData(data.to('cpu'), graph, vals=sets.squeeze(-1).detach().cpu(), dense=False,seed=dirac, nodecolor=True,edgecolor=False,seedhops=True,hoplabels=True,binarycut=False)
            ax2 = f1.add_subplot(122)
            g1,g2 = drawGraphFromData(data.to('cpu'), graph, vals=probabilities.detach().cpu(), dense=False,seed=dirac, nodecolor=True,edgecolor=False,seedhops=True,hoplabels=True,binarycut=False, clique = True)

            clique_size = len(list(max_clique(g1)))
#              print("NX Clique size is: ", clique_size)
#              print("True clique number: ", graph_clique_number(g1))
        
        
        total_index += num_nodes
        

    expected_weight_G = scatter_add(blank_sets[no_loop_col]*blank_sets[no_loop_row], batch[no_loop_row], 0, dim_size = num_graphs)
    set_cardinality = scatter_add(blank_sets, batch, 0 , dim_size = num_graphs)
    #print(set_cardinality)
    return blank_sets, expected_weight_G.detach(), max_cardinalities

In [20]:
#GET GUROBI GROUND TRUTHS
from  cut_utils import solve_gurobi_maxclique

import gurobipy as gp
from gurobipy import GRB

test_data_clique = []
for data in testdata:
    my_graph = to_networkx(Data(x=data.x, edge_index = data.edge_index)).to_undirected()
    print(my_graph)
    cliqno, _ = solve_gurobi_maxclique(my_graph, 500)
    data.clique_number = cliqno
    test_data_clique += [data]


Using license file /home/karalias/gurobi.lic
Academic license - for non-commercial use only





































































































































































































In [21]:
batch_size = 32
#test_data = traindata
test_data = testdata
#shuffle(test_data)
test_loader = DataLoader(test_data, batch_size, shuffle=False)
net.to(device)
count = 1
#Evaluation
net.eval()

gnn_nodes = []
gnn_edges = []
gnn_sets = {}
max_samples = 1
gnn_times = []
num_samples = max_samples
t_start = time.time()


for data in test_loader:

    num_graphs = data.batch.max().item()+1
    bestset = {}
    bestedges = np.zeros((num_graphs))
    maxset = np.zeros((num_graphs))

    total_samples = []
    for graph in range(num_graphs):
        curr_inds = (data.batch==graph)
        g_size = curr_inds.sum().item()
        if max_samples <= g_size: 
            samples = np.random.choice(curr_inds.sum().item(),max_samples, replace=False)
        else:
            samples = np.random.choice(curr_inds.sum().item(),max_samples, replace=True)

        total_samples +=[samples]

    data = data.to(device)

    t_0 = time.time()
    
#     max_size = data.x.shape[0]
#     #print(max_size)
#     if (max_size <= max_samples):
#             num_samples = max_size
#     else:
#             num_samples = max_samples
#     sample_locations = np.random.choice(max_size,max_samples, replace=False)

    for k in range(num_samples):
        t_datanet_0 = time.time()
        data_prime = get_diracs(data.to(device), 1, sparse = True, effective_volume_range=0.15, receptive_field = 7)
  
        initial_values = data_prime.x.detach()
        data_prime.x = torch.zeros_like(data_prime.x)
        g_offset = 0
        for graph in range(num_graphs):
            curr_inds = (data_prime.batch==graph)
            g_size = curr_inds.sum().item()
            graph_x = data_prime.x[curr_inds]
            data_prime.x[total_samples[graph][k] + g_offset]=1.
            g_offset += g_size
            
        retdz = net(data_prime)
        
        t_datanet_1 = time.time() - t_datanet_0
        print("data prep and fp: ", t_datanet_1)
        t_derand_0 = time.time()


        sets, set_edges, set_cardinality = derandomize_clique_final_speed(data_prime,(retdz["output"][0]), weight_factor =0.,draw=False, beam = 1)

        t_derand_1 = time.time() - t_derand_0
        print("Derandomization time: ", t_derand_1)


        for j in range(num_graphs):
            indices = (data.batch == j)
            if (set_cardinality[j]>maxset[j]):
                    maxset[j] = set_cardinality[j].item()
                    bestset[str(j)] = sets[indices].cpu()
                    bestedges[j] = set_edges[j].item()

        #del retdz
       # del data_prime
    t_1 = time.time()-t_0
    print("Current batch: ", count)
    print("Time so far: ", time.time()-t_0)
    gnn_sets[str(count)] = bestset
    
    gnn_nodes += [maxset]
    gnn_edges += [bestedges]
    gnn_times += [t_1]
    #del sets




    count += 1
    #if count > 200:
        #break
t_1 = time.time()
total_time = t_1 - t_start
print("Average time per graph: ", total_time/(len(test_data)))
#print(prof.key_averages().table(sort_by="self_cpu_time_total"))

data prep and fp:  0.19528794288635254
Derandomization time:  0.6902110576629639
Current batch:  1
Time so far:  0.8925487995147705
data prep and fp:  0.1503159999847412
Derandomization time:  0.8023681640625
Current batch:  2
Time so far:  0.9594218730926514
data prep and fp:  0.15389084815979004
Derandomization time:  0.768791675567627
Current batch:  3
Time so far:  0.930201530456543
data prep and fp:  0.1518864631652832
Derandomization time:  0.978672981262207
Current batch:  4
Time so far:  1.1376447677612305
data prep and fp:  0.15407228469848633
Derandomization time:  0.7829701900482178
Current batch:  5
Time so far:  0.9444413185119629
data prep and fp:  0.1525895595550537
Derandomization time:  0.7348346710205078
Current batch:  6
Time so far:  0.8961632251739502
data prep and fp:  0.03475189208984375
Derandomization time:  0.06750249862670898
Current batch:  7
Time so far:  0.10378599166870117
Average time per graph:  0.03027577546178078


In [23]:
#FLATTEN
flat_list = [item for sublist in gnn_edges for item in sublist]
for k in range(len(flat_list)):
    flat_list[k] = flat_list[k].item()
gnn_edges = (flat_list)

flat_list = [item for sublist in gnn_nodes for item in sublist]
for k in range(len(flat_list)):
    flat_list[k] = flat_list[k].item()
gnn_nodes = (flat_list)

In [24]:
tests = test_data_clique
ratios = [gnn_nodes[i]/tests[i].clique_number for i in range(len(tests))]
print(f"Mean ratio: {(np.array(ratios)).mean()} +/-  {(np.array(ratios)).std()}")

Mean ratio: 0.8387401619438469 +/-  0.19620118617274399
